In [ ]:
#pip install tensorflow

In [1]:
from tensorflow.keras import models, layers, utils, backend as K
import matplotlib.pyplot as plt
import shap
import tensorflow as tf
import numpy as np
import pandas as pd

In [2]:
print(tf.__version__)

2.4.0


### Simple Constants

In [3]:
hello = tf.constant('Hello World')

In [4]:
type(hello)

tensorflow.python.framework.ops.EagerTensor

In [5]:
x = tf.constant(100)

In [6]:
type(x)

tensorflow.python.framework.ops.EagerTensor

### Functions have Replaced Sessions (TF2 v TF1)

Examples below are from: https://www.tensorflow.org/guide/function

In [7]:
@tf.function
def add(a, b):
    return a + b

add(tf.ones([2,2]), tf.ones([2,2]))

Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


<tf.Tensor: shape=(2, 2), dtype=float32, numpy=
array([[2., 2.],
       [2., 2.]], dtype=float32)>

Note: Functions can be faster than eager code, especially for graphs with many small ops. But for graphs with a few expensive ops (e.g., convolutions) there may not be much of a time difference.

In [8]:
import timeit
conv_layer = tf.keras.layers.Conv2D(100,3)

@tf.function
def conv_fn(image):
    return conv_layer(image)

image = tf.zeros([1, 200, 200, 100])
#Warm up
conv_layer(image); conv_fn(image)
print("Eager conv:", timeit.timeit(lambda: conv_layer(image), number = 10))
print("Function conv:", timeit.timeit(lambda: conv_fn(image), number = 10))
print("Note how there's not much difference in performance for convolutions")

Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Eager conv: 0.6368969000000106
Function conv: 0.5236745000000269
Note how there's not much difference in performance for convolutions


### MNIST Image Data Set

https://www.tensorflow.org/datasets/keras_example#load_a_dataset

In [9]:
#pip install tensorflow-datasets & tfds-nightly

In [10]:
import tensorflow_datasets as tfds

In [11]:
#import data
(ds_train, ds_test), ds_info = tfds.load(
    'mnist', #name of dataset
    split = ['train','test'], #train/test split
    shuffle_files = True, #good to shuffle training data especially when using large datasets
    as_supervised = True, #returns a tuple instead of a dictionary
    with_info = True,
)

local data directory. If you'd instead prefer to read directly from our public
GCS bucket (recommended if you're running on GCP), you can instead set
data_dir=gs://tfds-data/datasets.





Dataset mnist downloaded and prepared to C:\Users\ianno\tensorflow_datasets\mnist\3.0.0. Subsequent calls will reuse this data.


In [12]:
#build training pipeline
def normalize_img(image, label):
    """Normalizes images: 'uint8' -> 'float32'."""
    return tf.cast(image, tf.float32) / 255., label

#TFDS provides images of type uint8 while model expects float32, this normalizes images
ds_train = ds_train.map(
    normalize_img, num_parallel_calls = tf.data.AUTOTUNE)
#as the dataset is being fit into memory, cache before shuffling for better performance
ds_train = ds_train.cache()
#for true randomness, set the shuffle buffer to full dataset size
ds_train = ds_train.shuffle(ds_info.splits['train'].num_examples)
#batch elements of the dataset after shuffling to get unique batches at each epoch
ds_train = ds_train.batch(128)
#good practice to end the pipeline by prefetching for performance
ds_train = ds_train.prefetch(tf.data.AUTOTUNE)

Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


In [13]:
#build evaluation pipeline
    #similar to training pipeline but don't need to call shuffle and caching
    #is done after batching because batches can be the same between epochs
ds_test = ds_test.map(
    normalize_img, num_parallel_calls=tf.data.AUTOTUNE)
ds_test = ds_test.batch(128)
ds_test = ds_test.cache()
ds_test = ds_test.prefetch(tf.data.AUTOTUNE)

In [14]:
#create and train the model
model = tf.keras.models.Sequential([
    tf.keras.layers.Flatten(input_shape=(28,28)), #transforms the format of the images (28x28) to a one-dimensional array
    tf.keras.layers.Dense(128, activation = 'relu'), #first Dense layer that has 128 nodes/neurons; recall we can play around with activiation method (e.g., relu)
    tf.keras.layers.Dense(10) #second Dense layer returns the desired output (i.e., 10 in this case for numbers 0-9)
])

In [15]:
#compile the model
model.compile(
    optimizer = tf.keras.optimizers.Adam(0.001),
    loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics = [tf.keras.metrics.SparseCategoricalAccuracy()],
)

In [16]:
#train and fit the model
model.fit(
    ds_train,
    epochs=6,
    validation_data=ds_test
)

Epoch 1/6
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
    463/Unknown - 5s 5s/step - loss: 2.4957 - sparse_categorical_accuracy: 0.062 - 5s 4ms/step - loss: 2.1164 - sparse_categorical_accuracy: 0.27 - 5s 4ms/step - loss: 1.8940 - sparse_categorical_accuracy: 0.39 - 5s 4ms/step - loss: 1.7103 - sparse_categorical_accuracy: 0.47 - 5s 4ms/step - loss: 1.5565 - sparse_categorical_accuracy: 0.53 - 5s 4ms/step - loss: 1.4363 - sparse_categorical_accuracy: 0.57 - 5s 4ms/step - loss: 1.3399 - sparse_categorical_accuracy: 0.60 - 5s 4ms/step - loss: 1.2615 - sparse_categorical_accuracy: 0.63 - 5s 4ms/step - loss: 1.1958 - sparse_categorical_accuracy: 0.65 - 5s 4ms/step - loss: 1.1437 - sparse_categorical_accuracy: 0.67 - 5s 4ms/step - loss: 1

Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
469/469 [==============================] - 9s 9ms/step - loss: 0.6289 - sparse_categorical_accuracy: 0.8229 - val_loss: 0.1900 - val_sparse_categorical_accuracy: 0.9455
Epoch 2/6
469/469 [==============================] - ETA: 37s - loss: 0.1400 - sparse_categorical_accuracy: 0.96 - ETA: 1s - loss: 0.1637 - sparse_categorical_accuracy: 0.9584 - ETA: 2s - loss: 0.1762 - sparse_categorical_accuracy: 0.955 - ETA: 1s - loss: 0.1869 - sparse_categorical_accuracy: 0.952 - ETA: 1s - loss: 0.1916 - sparse_categorical_accuracy: 0.951 - ETA: 1s - loss: 0.1937 - sparse_categorical_accuracy: 0.950 - ETA: 1s - loss: 0.1942 - sparse_categorical_accuracy: 0.949 - ETA: 1s - loss: 0.1941 - sparse_

469/469 [==============================] - ETA: 34s - loss: 0.0953 - sparse_categorical_accuracy: 0.96 - ETA: 1s - loss: 0.0837 - sparse_categorical_accuracy: 0.9743 - ETA: 1s - loss: 0.0793 - sparse_categorical_accuracy: 0.976 - ETA: 1s - loss: 0.0774 - sparse_categorical_accuracy: 0.977 - ETA: 1s - loss: 0.0766 - sparse_categorical_accuracy: 0.978 - ETA: 1s - loss: 0.0761 - sparse_categorical_accuracy: 0.978 - ETA: 1s - loss: 0.0761 - sparse_categorical_accuracy: 0.978 - ETA: 1s - loss: 0.0758 - sparse_categorical_accuracy: 0.978 - ETA: 1s - loss: 0.0757 - sparse_categorical_accuracy: 0.978 - ETA: 1s - loss: 0.0755 - sparse_categorical_accuracy: 0.978 - ETA: 1s - loss: 0.0754 - sparse_categorical_accuracy: 0.978 - ETA: 1s - loss: 0.0752 - sparse_categorical_accuracy: 0.978 - ETA: 1s - loss: 0.0751 - sparse_categorical_accuracy: 0.978 - ETA: 1s - loss: 0.0749 - sparse_categorical_accuracy: 0.978 - ETA: 1s - loss: 0.0747 - sparse_categorical_accuracy: 0.978 - ETA: 1s - loss: 0.0746 - s